In [5]:
%pip install pydub

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os._exit(00)

In [73]:
import os

import librosa
import pydub
import audioop

import torch
import torchaudio

import numpy as np

from utils.dataset_preprocessing import 

SyntaxError: invalid syntax (2907836060.py, line 12)

In [2]:
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
data_processed_dir = os.path.join(project_dir, 'gdsc_data/data_processed/')
data_source_dir = os.path.join(project_dir, 'gdsc_data/data_source/')

DATA_TAG = "data_8c86715"

In [3]:
#final_dataset_path = find_final_dataset_path(data_processed_dir, DATA_TAG)
final_dataset_path_augmented = os.path.join(data_processed_dir, DATA_TAG, '01_apply_ir_function')
final_dataset_val_as_train_path_augmented = os.path.join(data_processed_dir, DATA_TAG, '03_apply_ir_function_on_validation')

final_dataset_path = os.path.join(data_processed_dir, DATA_TAG, '04_split_original_into_windows')
final_dataset_val_as_train_path = os.path.join(data_processed_dir, DATA_TAG, '05_split_resampled_validation_into_windows/')

In [4]:
train_dir_augmented = os.path.join(final_dataset_path_augmented, 'train/')
val_as_train_dir_augmented = os.path.join(final_dataset_val_as_train_path_augmented, 'train/')

train_dir = os.path.join(final_dataset_path, 'train/')
val_as_train_dir = os.path.join(final_dataset_val_as_train_path, 'train/')

val_dir = os.path.join(final_dataset_val_as_train_path, 'val/')

In [5]:
etalon_path = os.path.join(data_source_dir, 'train/', 'Chorthippusbiguttulus_CHOBIG_20190906_164243.M.wav')

etalon_signal_torch, SR = torchaudio.load(etalon_path)
etalon_signal_pydub = pydub.AudioSegment.from_file(etalon_path)

## Match amplitude in pydub

In [6]:
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    print(sound.dBFS)
    print(change_in_dBFS)
    print(sound.rms)
    return sound.apply_gain(change_in_dBFS)

### Save
Save to temp for comparison with original torch file

In [7]:
match_target_amplitude(etalon_signal_pydub, -20).export('/root/data/tmp/scaled.wav', format='wav')

-43.7414066304196
23.741406630419597
213


<_io.BufferedRandom name='/root/data/tmp/scaled.wav'>

## Load in torch, resave and scale with pydub

In [8]:
torchaudio.save('/root/data/tmp/torch_PCM_F.wav', etalon_signal_torch, sample_rate=SR)

In [9]:
pydub.utils.mediainfo('/root/data/tmp/torch_PCM_F.wav')

{'index': '0',
 'codec_name': 'pcm_f32le',
 'codec_long_name': 'PCM 32-bit floating point little-endian',
 'profile': 'unknown',
 'codec_type': 'audio',
 'codec_tag_string': '[3][0][0][0]',
 'codec_tag': '0x0003',
 'sample_fmt': 'flt',
 'sample_rate': '44100',
 'channels': '1',
 'channel_layout': 'unknown',
 'bits_per_sample': '32',
 'id': 'N/A',
 'r_frame_rate': '0/0',
 'avg_frame_rate': '0/0',
 'time_base': '1/44100',
 'start_pts': 'N/A',
 'start_time': 'N/A',
 'duration_ts': '293077',
 'duration': '6.645737',
 'bit_rate': '1411269',
 'max_bit_rate': 'N/A',
 'bits_per_raw_sample': 'N/A',
 'nb_frames': 'N/A',
 'nb_read_frames': 'N/A',
 'nb_read_packets': 'N/A',
 'DISPOSITION': {'default': '0',
  'dub': '0',
  'original': '0',
  'comment': '0',
  'lyrics': '0',
  'karaoke': '0',
  'forced': '0',
  'hearing_impaired': '0',
  'visual_impaired': '0',
  'clean_effects': '0',
  'attached_pic': '0',
  'timed_thumbnails': '0',
  'captions': '0',
  'descriptions': '0',
  'metadata': '0',
  'de

In [ ]:
etalon_signal_pydub.sample_width

In [10]:
torch_etalon_pydubed = pydub.AudioSegment.from_file('/root/data/tmp/torch_PCM_F.wav')
match_target_amplitude(torch_etalon_pydubed, -20).export('/root/data/tmp/torch_PCM_F_pydub_set_amplitude.wav', format="wav")

-43.7194744089578
23.7194744089578
13994460


<_io.BufferedRandom name='/root/data/tmp/torch_PCM_F_pydub_set_amplitude.wav'>

In [32]:
torch_etalon_pydubed.sample_width

4

The goal is, when we load the two (converted with pydub and with our code) - they should be equal

In [2]:
rms = torch.sqrt(((etarmson_signal_torch ** 2).mean(dim=1)))

NameError: name 'torch' is not defined

In [ ]:
source_dBFS = 20 * torch.log10(rms)
source_dBFS

In [79]:
class AmplitudeScaler():
    def __init__(self, target_dBFS=-20):
        """Only works with float32 torch tensors"""
        
        self.target_dBFS = target_dBFS         
        
    def __call__(self, signal):
        # Flat tensor, 
        if len(signal.shape) == 1:
            _dim = 0
        elif len(signal.shape) == 2:
        # (CH, SAMPLES)
            _dim=1
            
        rms = torch.sqrt((signal**2).mean(dim=_dim))
        
        source_dBFS = (20 * torch.log10(rms))
        delta_dBFS = self.target_dBFS - source_dBFS
        
        #gain in real values
        gain = 10 ** (delta_dBFS / 20)
        signal = signal * gain
        
        return signal

In [28]:
pydub.utils.mediainfo('/root/data/tmp/torch_PCM_F_pydub_set_amplitude.wav')

{'index': '0',
 'codec_name': 'pcm_s32le',
 'codec_long_name': 'PCM signed 32-bit little-endian',
 'profile': 'unknown',
 'codec_type': 'audio',
 'codec_tag_string': '[1][0][0][0]',
 'codec_tag': '0x0001',
 'sample_fmt': 's32',
 'sample_rate': '44100',
 'channels': '1',
 'channel_layout': 'unknown',
 'bits_per_sample': '32',
 'id': 'N/A',
 'r_frame_rate': '0/0',
 'avg_frame_rate': '0/0',
 'time_base': '1/44100',
 'start_pts': 'N/A',
 'start_time': 'N/A',
 'duration_ts': '293077',
 'duration': '6.645737',
 'bit_rate': '1411252',
 'max_bit_rate': 'N/A',
 'bits_per_raw_sample': '32',
 'nb_frames': 'N/A',
 'nb_read_frames': 'N/A',
 'nb_read_packets': 'N/A',
 'DISPOSITION': {'default': '0',
  'dub': '0',
  'original': '0',
  'comment': '0',
  'lyrics': '0',
  'karaoke': '0',
  'forced': '0',
  'hearing_impaired': '0',
  'visual_impaired': '0',
  'clean_effects': '0',
  'attached_pic': '0',
  'timed_thumbnails': '0',
  'captions': '0',
  'descriptions': '0',
  'metadata': '0',
  'dependent':

In [63]:
rms = torch.sqrt((etalon_signal_torch ** 2).mean())

dBFS_delta = -20 - 20 * torch.log10(rms) 

tensor(23.7195)

In [68]:
torchaudio.save('/root/data/tmp/torch_etalon_scaled.wav', etalon_signal_torch * 10**(23.7195/20), SR)

In [69]:
etalon_scaled1 = torchaudio.load('/root/data/tmp/torch_etalon_scaled.wav')[0]
etalon_scaled2 = torchaudio.load('/root/data/tmp/torch_PCM_F_pydub_set_amplitude.wav')[0]

In [76]:
etalon_scaled1

tensor([[-0.0576, -0.1072, -0.0955,  ..., -0.1021, -0.0951, -0.1063]])

In [83]:
AmplitudeScaler()(etalon_signal_torch) - etalon_scaled2

tensor([[ 0.0000e+00,  0.0000e+00,  7.4506e-09,  ..., -3.7253e-08,
         -7.4506e-09, -2.9802e-08]])

In [72]:
 etalon_scaled2

tensor([[-0.0576, -0.1072, -0.0955,  ..., -0.1021, -0.0951, -0.1063]])